# 0. INSTALL DEPENDENCIES

In [13]:
# %pip install tensorflow # installed via conda...
%pip install gym
%pip install keras
%pip install keras-rl2
%pip install pygame

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached keras_rl2-1.0.5-py3-none-any.whl (52 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 697.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 618.9 kB/s eta 0:00:0000:0100:02
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# 1. TEST RANDOM ENVIRONMENT WITH OPENAI GYM

In [5]:
import gym
import random
import pygame

In [6]:
env = gym.make('CartPole-v1', render_mode="human")
states = env.observation_space.shape[0]
actions = env.action_space.n

In [7]:
actions

2

In [8]:
states

4

In [47]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = random.choice([0,1])
        # n_state, reward, done, info = env.step(action)
        n_state, reward, terminated, truncated, info = env.step(action)
        score+=reward
        done = terminated or truncated
    print('Episode: {} Score:{}'.format(episode, score))

/Users/dylangraham/miniconda3/envs/my_env/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode: 1 Score:16.0
Episode: 2 Score:54.0
Episode: 3 Score:28.0
Episode: 4 Score:29.0
Episode: 5 Score:23.0
Episode: 6 Score:12.0
Episode: 7 Score:14.0
Episode: 8 Score:12.0
Episode: 9 Score:12.0
Episode: 10 Score:24.0


# 2. CREATE DEEP LEARNING MODEL WITH KERAS

In [9]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
# from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.legacy import Adam

In [31]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1, states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [32]:
model = build_model(states, actions)

In [33]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 4)                 0         
                                                                 
 dense_9 (Dense)             (None, 24)                120       
                                                                 
 dense_10 (Dense)            (None, 24)                600       
                                                                 
 dense_11 (Dense)            (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________



# 3. BUILD AGENT WITH KERAS-RL



In [34]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [35]:
# def build_agent(model, actions):
#     policy = BoltzmannQPolicy()
#     memory = SequentialMemory(limit = 50000, window_length = 1)
#     agent = DQNAgent(model = model, memory = memory, policy = policy, nb_actions = actions, nb_steps_warmup = 10, target_model_update = 1e-2 )
#     return agent

def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn


In [36]:
# agent = build_agent(model, actions)
# agent.compile(Adam(learning_rate=1e-3), metrics=['mae'])
# agent.fit(env, nb_steps=50000, visualize=False, verbose=1)

dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

/Users/dylangraham/miniconda3/envs/my_env/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2023-05-02 14:24:30.814266: W tensorflow/c/c_api.cc:291] Operation '{name:'dense_11/bias/Assign' id:1650 op device:{requested: '', assigned: ''} def:{{{node dense_11/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_11/bias, dense_11/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Training for 50000 steps ...
Interval 1 (0 steps performed)


ValueError: Error when checking input: expected flatten_3_input to have shape (1, 4) but got array with shape (1, 2)

# 4. RELOADING AGENT FROM MEMORY